# Risk Assessment for Lending Club Loans

This is the analysis notebook for this project.

See the README.md file for background information about this project and the details of the used data.

**Objective:** Predict whether a requested loan will be paid back in full or not (i.e. will be charged off) to help investors choose where to invest.

Loading the libraries used by this notebook:

In [1]:
# load libraries
import pandas as pd
import numpy

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Reading and Cleaning the Data

Load data set and do some initial cleanup.

In [2]:
# clean up downloaded data
#
# read data and skip first row because it includes no useful information
loans_2007 = pd.read_csv('data/LoanStats3a_2007_2011.csv', skiprows=1)

# drop columns with more than half entries NAN
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)

# drop columns 'desc' (text description of loan) and 'url' (link to loan, requires investor account)
if 'desc' in loans_2007.columns:
    loans_2007 = loans_2007.drop(['desc'],axis=1)
if 'url' in loans_2007.columns:
    loans_2007 = loans_2007.drop(['url'],axis=1)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Drop columns which contain information that is 
* 'from the future' on how the loan was repaid (or not),
* redundant (e.g. first 3 digits of ZIP code yield same information as 'state' column),
* not relevant to credit risk assessment (e.g. 'member_id'),
* tricky to extract meaningful information from,
* NAN for a large number of rows, or
* identical in all rows.

In [3]:
drop_columns = [
#    'id',
#    'member_id',
    'funded_amnt',
    'funded_amnt_inv',
    'grade',
    'sub_grade',
    'emp_title',
    'issue_d',
    'zip_code',
    'out_prncp',
    'out_prncp_inv',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_prncp',
    'total_rec_int',
    'total_rec_late_fee',
    'recoveries',
    'collection_recovery_fee',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'pub_rec_bankruptcies',
    'last_credit_pull_d',
    'addr_state',
    'title',
    'earliest_cr_line'
]

loans_2007.drop( drop_columns, axis=1, inplace=True )

# find and remove single-value columns ##
drop_columns_single = []

for col in loans_2007.columns:
    unique_non_null = loans_2007[col].dropna().unique()
    # print(col,'-->',len(unique_non_null))
    if ( len(unique_non_null) == 1 ):
        drop_columns.append( col )
        
loans_2007.drop( drop_columns_single, axis=1, inplace=True )

Remove all rows with NAN entries. Print number of NAN in each column to confirm.

In [4]:
# drop rows with NAN
loans_2007.dropna(inplace=True)

# print number of NAN values in each column
print( loans_2007.isnull().sum() )

loan_amnt                     0
term                          0
int_rate                      0
installment                   0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
loan_status                   0
pymnt_plan                    0
purpose                       0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
collections_12_mths_ex_med    0
policy_code                   0
application_type              0
acc_now_delinq                0
chargeoff_within_12_mths      0
delinq_amnt                   0
tax_liens                     0
hardship_flag                 0
disbursement_method           0
debt_settlement_flag          0
dtype: int64


The column 'loan_status' is the target column, i.e. this is the loan status we want to predict. Look at the values and counts in this column:

In [5]:
print( loans_2007['loan_status'].value_counts() )

Fully Paid                                             33186
Charged Off                                             5419
Does not meet the credit policy. Status:Fully Paid      1895
Does not meet the credit policy. Status:Charged Off      723
Name: loan_status, dtype: int64


Keep only two loan status cases- 'Fully Paid' and 'Charged Off'. Replace the status string with '1' (= "Fully Paid") and '0' (= "Charged Off"). Finally, drop columns that have the same value in all rows.

In [6]:
# keep only two status cases
loans_2007 = loans_2007[
    (loans_2007['loan_status'] == 'Fully Paid') |
    (loans_2007['loan_status'] == 'Charged Off')
]

# use dict to replace status with 1/0 identifier
mapping_dict = {
    "loan_status": {
        "Fully Paid": 1,
        "Charged Off": 0
    }
}
loans_2007 = loans_2007.replace(mapping_dict)

# print target column value counts
print( loans_2007['loan_status'].value_counts() )

# save cleaned-up data as .csv file for later use
loans_2007.to_csv('data/filtered_loans_2007.csv', index=False)

1    33186
0     5419
Name: loan_status, dtype: int64


# Selecting Features For Risk Analysis

Read data clened up in __Part 1__ into dataframe.

In [7]:
# read cleaned-up data set into dataframe
loans = pd.read_csv('data/filtered_loans_2007.csv')

print( loans.dtypes.value_counts() )

float64    16
object     13
int64       1
dtype: int64


In [8]:
# look at text columns (data type 'object')
object_columns_df = loans.select_dtypes( include=['object'] )
print( object_columns_df.head(1) )

         term int_rate emp_length home_ownership verification_status  \
0   36 months   10.65%  10+ years           RENT            Verified   

  pymnt_plan      purpose revol_util initial_list_status application_type  \
0          n  credit_card      83.7%                   f       Individual   

  hardship_flag disbursement_method debt_settlement_flag  
0             N                Cash                    N  


In [9]:
# display first 5 categorical columns
cols = ['home_ownership', 'verification_status', 'emp_length', 'term']

for col in cols:
    print( "---->"+col)
    print( loans[col].value_counts() )

---->home_ownership
RENT        18456
MORTGAGE    17218
OWN          2834
OTHER          96
NONE            1
Name: home_ownership, dtype: int64
---->verification_status
Not Verified       16418
Verified           12379
Source Verified     9808
Name: verification_status, dtype: int64
---->emp_length
10+ years    8895
< 1 year     4560
2 years      4382
3 years      4090
4 years      3433
5 years      3280
1 year       3231
6 years      2226
7 years      1770
8 years      1481
9 years      1257
Name: emp_length, dtype: int64
---->term
 36 months    28187
 60 months    10418
Name: term, dtype: int64


In [10]:
# check reasons for the loan
cols = ['purpose']

for col in cols:
    print( loans[col].value_counts() )

debt_consolidation    18252
credit_card            4992
other                  3821
home_improvement       2875
major_purchase         2110
small_business         1780
car                    1496
wedding                 933
medical                 668
moving                  555
house                   368
vacation                349
educational             311
renewable_energy         95
Name: purpose, dtype: int64


In [11]:
# categorical columns
#
# replace employment length with numeric values
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans.replace( mapping_dict, inplace=True )

# strip '%' from int_rate and revol_util and convert to float
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')

# create dummy variables for categorical columns
cols = [
    'home_ownership',
    'verification_status',
    'purpose',
    'term'  
]

# returns a new Dataframe containing 1 column for each dummy variable
dummy_df = pd.get_dummies(loans[cols])

# use the concat method to add these dummy columns back to the original Dataframe:
loans = pd.concat([loans, dummy_df], axis=1)

# drop the original columns
loans = loans.drop(cols, axis=1)

# drop remaining 'object' type columns
object_columns_df = loans.select_dtypes( include=['object'] )
loans = loans.drop(object_columns_df, axis=1)

# check data types in data frame:
print( loans.dtypes.value_counts() )

uint8      24
float64    18
int64       2
dtype: int64


# Credit Risk Analysis with Machine Learning and Prediction

In [12]:
# print dataframe info
print( loans.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38605 entries, 0 to 38604
Data columns (total 44 columns):
loan_amnt                              38605 non-null float64
int_rate                               38605 non-null float64
installment                            38605 non-null float64
emp_length                             38605 non-null int64
annual_inc                             38605 non-null float64
loan_status                            38605 non-null int64
dti                                    38605 non-null float64
delinq_2yrs                            38605 non-null float64
inq_last_6mths                         38605 non-null float64
open_acc                               38605 non-null float64
pub_rec                                38605 non-null float64
revol_bal                              38605 non-null float64
revol_util                             38605 non-null float64
total_acc                              38605 non-null float64
collections_12_mths_ex_med 

### Baseline: Predict all loans will be paid off on time

In [13]:
# Predict that all loans will be paid off on time.
predictions = pd.Series(numpy.ones(loans.shape[0]))

tn = len( loans[ (predictions == 0) & (loans["loan_status"] == 0) ] )
tp = len( loans[ (predictions == 1) & (loans["loan_status"] == 1) ] )
fn = len( loans[ (predictions == 0) & (loans["loan_status"] == 1) ] )
fp = len( loans[ (predictions == 1) & (loans["loan_status"] == 0) ] )

# false positive rate
fpr = fp / (fp + tn)

# true positive rate
tpr = tp / ( tp + fn )

print( "False Positive rate: ", fpr )
print( "True  Positive rate: ", tpr )

False Positive rate:  1.0
True  Positive rate:  1.0


### Logistic Regression

In [14]:
# logistic regression
lr = LogisticRegression()

features = loans.drop('loan_status', axis=1)
target = loans['loan_status']

lr.fit( features, target )

predictions = lr.predict( features )

# cross validation
lr = LogisticRegression()
kf = KFold(features.shape[0], random_state=1)

predictions = cross_val_predict( lr, features, target, cv=kf )
predictions = pd.Series( predictions )

tn = len( loans[ (predictions == 0) & (loans["loan_status"] == 0) ] )
tp = len( loans[ (predictions == 1) & (loans["loan_status"] == 1) ] )
fn = len( loans[ (predictions == 0) & (loans["loan_status"] == 1) ] )
fp = len( loans[ (predictions == 1) & (loans["loan_status"] == 0) ] )

# false positive rate
fpr = fp / (fp + tn)

# true positive rate
tpr = tp / ( tp + fn )

print( "False Positive rate: ", fpr )
print( "True  Positive rate: ", tpr )

False Positive rate:  0.9990773205388448
True  Positive rate:  0.9992466702826494


### Logistic Regression with Penalties

In [15]:
# logistic regression + penalizing the classifier
lr = LogisticRegression( class_weight = 'balanced' )
kf = KFold(features.shape[0], random_state=1)

predictions = cross_val_predict( lr, features, target, cv=kf )
predictions = pd.Series( predictions )

tn = len( loans[ (predictions == 0) & (loans["loan_status"] == 0) ] )
tp = len( loans[ (predictions == 1) & (loans["loan_status"] == 1) ] )
fn = len( loans[ (predictions == 0) & (loans["loan_status"] == 1) ] )
fp = len( loans[ (predictions == 1) & (loans["loan_status"] == 0) ] )

# false positive rate
fpr = fp / (fp + tn)

# true positive rate
tpr = tp / ( tp + fn )

print( "False Positive rate: ", fpr )
print( "True  Positive rate: ", tpr )

False Positive rate:  0.38881712493079906
True  Positive rate:  0.6575965768697644


### Logistic Regression with Manual Penalties

In [16]:
# logistic regression + manual penalties
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression( class_weight = penalty )
kf = KFold(features.shape[0], random_state=1)

predictions = cross_val_predict( lr, features, target, cv=kf )
predictions = pd.Series( predictions )

tn = len( loans[ (predictions == 0) & (loans["loan_status"] == 0) ] )
tp = len( loans[ (predictions == 1) & (loans["loan_status"] == 1) ] )
fn = len( loans[ (predictions == 0) & (loans["loan_status"] == 1) ] )
fp = len( loans[ (predictions == 1) & (loans["loan_status"] == 0) ] )

# false positive rate
fpr = fp / (fp + tn)

# true positive rate
tpr = tp / ( tp + fn )

print( "False Positive rate: ", fpr )
print( "True  Positive rate: ", tpr )

False Positive rate:  0.09134526665436428
True  Positive rate:  0.2447718917615862


### Random Forests

In [17]:
# random forests + penalties
penalty = {
    0: 10,
    1: 1
}

#lr = RandomForestClassifier(class_weight='balanced', random_state=1)
lr = RandomForestClassifier(class_weight=penalty, random_state=1)
kf = KFold(features.shape[0], random_state=1)

predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print( "False Positive rate: ", fpr )
print( "True  Positive rate: ", tpr )

False Positive rate:  0.9466691271452298
True  Positive rate:  0.9773398421020912


# Conclusion
The Logistic Regression (with manual penalties) achieves 25% true positive rate at 9% false positive rate. This is the lowest false positive rate for all compared algorithms, so this seems like the best choice when wanting to minimize loss of money to loans that are not being paid back in full.